In [10]:
from acdc.ioi.utils import get_gpt2_small

model = get_gpt2_small()

Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda


In [11]:
import acdc.transprop.utils as tp
import importlib
importlib.reload(tp)
transprop_things = tp.get_transprop_things(20, "frac_correct")
data = tp.get_prompt_data(5, model)

Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda
torch.Size([10, 50257])


In [18]:
import torch

ds = transprop_things.validation_data
model = transprop_things.tl_model
output = transprop_things.tl_model(ds)
indices = torch.sum(transprop_things.validation_mask, dim=1)
logits = torch.stack([
    torch.squeeze(output[i, indices[i] - 1, :]) for i in range(indices.shape[0])
])
print(logits.shape)
top_indices = torch.topk(logits, k=5, dim=-1).indices[9]
print(top_indices)
predicted_words = [model.tokenizer.decode([idx.item()]) for idx in top_indices]
print(predicted_words)
print(model.tokenizer.decode(ds[9]))
print(transprop_things.validation_labels[9])
print(transprop_things.validation_metric(output))
print(transprop_things.validation_labels[9])
print(model.tokenizer.decode(transprop_things.validation_labels[9]))

model.eval()

from torch.nn import CrossEntropyLoss
loss_fn = CrossEntropyLoss()
loss = loss_fn(logits, transprop_things.validation_labels)
wrong_loss = loss_fn(logits, transprop_things.validation_labels)
print("Cross entropy loss", loss, wrong_loss)
# # Define a prompt
prompt = "If all apples are fruits and all fruits are edible, then it can be inferred that all apples are"

# # Tokenize the prompt
input_ids = model.tokenizer.encode(prompt, return_tensors="pt")

# # Generate the next word
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs[:, -1, :]  # Take the logits of the last position
    top_indices = torch.topk(logits, k=5, dim=-1).indices[0]


# # Decode the predicted index to get the next word
predicted_words = [model.tokenizer.decode([idx.item()]) for idx in top_indices]

print("Next word prediction:", predicted_words)

torch.Size([10, 50257])
tensor([31853,  1974,   262,   257,   275], device='cuda:0')
[' barric', ' comb', ' the', ' a', ' b']
banana implies comb and if comb then barricade therefore by the transitive property banana also implies<|endoftext|><|endoftext|>
tensor(31853, device='cuda:0')
torch.Size([10, 21, 50257])
torch.Size([10])
torch.Size([10])
tensor([-2.5023, -7.2054, -0.3910, -1.9623, -2.8029, -3.7070,  0.1475, -2.6524,
        12.7613, -3.7197], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([-3.1773, -2.0016, -3.9363, -2.1811, -4.2002, -1.3575,  0.1300,  0.5678,
        16.3911, -1.0792], device='cuda:0', grad_fn=<IndexBackward0>)
tensor(-0.5000, device='cuda:0')
tensor(31853, device='cuda:0')
 barric
Cross entropy loss tensor(3.4708, device='cuda:0', grad_fn=<NllLossBackward0>)
Next word prediction: [' edible', ' fruits', ' fruit', ' not', ' the']


In [13]:
import acdc.transprop.utils as tp
import importlib
importlib.reload(tp)
from acdc.ioi.utils import get_gpt2_small
import gc
def perform_baseline_evaluation(metrics, model, prompts):
    results = {}
    for prompt in prompts:
        metric_results = {}
        for metric in metrics:
            transprop_things = tp.get_transprop_things(20,
                                                    metric,    
                                                    model=model,
                                                    prompt=prompt)
            ds = transprop_things.validation_data
            output = transprop_things.tl_model(ds)
            metric_result = transprop_things.validation_metric(output)
            metric_results[metric] = metric_result
            model.reset_hooks()
        results[prompt] = metric_results
    return results

In [14]:
model = get_gpt2_small()

Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda


In [15]:
prompt_templates = [
    "{a} implies {b} and if {b} then {c} therefore by the transitive property {a} also implies",
    "if all {a} are {b} and {b} are {c} then all {a} are",
    "if all {a} are {b} and {b} is a type of {c} then it can be inferred that all {a} are a type of",
    "{a} implies {b} and if {b} then {c} therefore {a} also implies",
    "{a} implies {b} and if {b} then {c} therefore by the transitive property {a} also implies",
    "{a} implies {b} and {b} implies {c} then by the transitive property {a} also implies",
    "{a} is a type of {b} and all {b} are {c} therefore {a} is also a type of"
]
baseline = perform_baseline_evaluation(["logit_diff", "frac_correct", "kl_div"], model, prompt_templates)
print(baseline)

torch.Size([10, 50257])
torch.Size([10, 21, 50257])
torch.Size([10])
torch.Size([10])
torch.Size([10, 50257])
torch.Size([10, 21, 50257])
torch.Size([10])
torch.Size([10])
tensor([-2.5023, -7.2054, -0.3910, -1.9623, -2.8029, -3.7070,  0.1475, -2.6524,
        12.7613, -3.7197], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([-3.1773, -2.0016, -3.9363, -2.1811, -4.2002, -1.3575,  0.1300,  0.5678,
        16.3911, -1.0792], device='cuda:0', grad_fn=<IndexBackward0>)
torch.Size([10, 50257])
torch.Size([10, 50257])
torch.Size([10, 17, 50257])
torch.Size([10])
torch.Size([10])
torch.Size([10, 50257])
torch.Size([10, 17, 50257])
torch.Size([10])
torch.Size([10])
tensor([-2.6249e+00, -7.3407e+00, -4.1402e-01, -2.0514e+00, -2.9313e+00,
        -3.7960e+00, -3.7928e-03, -2.7974e+00, -2.7686e+00, -3.8357e+00],
       device='cuda:0', grad_fn=<IndexBackward0>)
tensor([-3.2544, -2.1323, -3.9307, -2.2850, -4.2473, -1.3740, -0.0423,  0.4165,
        -1.6351, -1.2688], device='cuda:0', grad_fn=<In